### osu!nn #6: Rhythm Predictor

Calculates a map's rhythm from the music and the timing.

Synthesis of "rhythmData"
* rhythmModel x 1
* momentumModel x 1
* timingData x 1
* (Music) x 1

Synthesis Time: ~2 seconds

Final edit: 2018/8/16

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, re

In [2]:
model = tf.keras.models.load_model(
    "saved_rhythm_model",
    custom_objects=None,
    compile=False
);
model.compile(loss='mse',
            optimizer=tf.train.RMSPropOptimizer(0.001),
            metrics=['mae']);

In [3]:
# root = "..\\osureader\\mapdata_test";
fn = "mapthis.npz";

def read_npz(fn):
    with np.load(fn) as data:
        wav_data = data["wav"];
        wav_data = np.swapaxes(wav_data, 2, 3);
        ticks = data["ticks"];
        timestamps = data["timestamps"];
        extra = data["extra"];
        
        # Extra vars
        bpms = extra[0];
        slider_lengths = extra[1];
        ex1 = (60000 / bpms) / 500 - 1;
        ex2 = bpms / 120 - 1;
        ex3 = slider_lengths / 150 - 1;
        
        div_data = np.array([[int(k%4==0), int(k%4==1), int(k%4==2), int(k%4==3), ex1[k], ex2[k], ex3[k]] for k in ticks]);
    return wav_data, div_data, ticks, timestamps;

test_data, div_data, ticks, timestamps = read_npz(fn);

In [4]:
# Make time intervals from test data
time_interval = 16;
if test_data.shape[0]%time_interval > 0:
    test_data = test_data[:-(test_data.shape[0]%time_interval)];
    div_data = div_data[:-(div_data.shape[0]%time_interval)];
test_data2 = np.reshape(test_data, (-1, time_interval, test_data.shape[1], test_data.shape[2], test_data.shape[3]))
div_data2 = np.reshape(div_data, (-1, time_interval, div_data.shape[1]))

test_predictions = model.predict([test_data2, div_data2]);
preds = test_predictions.reshape(-1, test_predictions.shape[2]);
divs = div_data2.reshape(-1, div_data2.shape[2]);
margin = np.expand_dims(- 0.0 * divs[:, 0] + 0.0 * divs[:, 1] - 0.0 * divs[:, 2] + 0.0 * divs[:, 3], 1);
is_obj_pred = (1 + np.sign(preds[:, 0:1] + margin)) / 2;
obj_type_pred = np.sign(preds[:, 1:4] - np.tile(np.expand_dims(np.max(preds[:, 1:4], axis=1), 1), (1, 3))) + 1;
others_pred = (1 + np.sign(preds[:, 4:test_predictions.shape[1]] + 0.5)) / 2;
another_pred_result = np.concatenate([is_obj_pred, is_obj_pred * obj_type_pred, others_pred], axis=1);

is_obj_pred = (1 + np.sign(preds[:, 0:1] + margin + -.65)) / 2;
print(np.sum(is_obj_pred) / preds.shape[0]);

0.244485294118


In [6]:
def load_momentum_minmax(fn):
    data = np.load(fn);
    return data;
mommax, mommin = load_momentum_minmax("momentum_minmax.npy");

momentum_model = tf.keras.models.load_model(
    "saved_rhythm_model_momentums",
    custom_objects=None,
    compile=False
);
momentum_model.compile(loss='mse',
            optimizer=tf.train.RMSPropOptimizer(0.001),
            metrics=['mae']);

In [7]:
momentum_predictions_output = momentum_model.predict([test_data2, div_data2]);
momentum_predictions = (momentum_predictions_output.reshape(-1, 2) + 1) / 2 / 0.8 * (mommax - mommin) + mommin;

This section will draw a demo map with the distances. It is used for debugging! delete later!

In [8]:
# note_positions = is_obj_pred[:, 0];
# prev_timestamp = 0;
# dist_len = 9e9;
# max_dist_len = 150;
# x = 256;
# y = 192;
# cnt = 0;
# prev_angle = 0;
# for i, k in enumerate(note_positions):
#     if k == 1:
#         time_dist = timestamps[i] - prev_timestamp;
#         dist_len = time_dist * momentum_predictions[i][0] / 2;
#         if dist_len > max_dist_len:
#             x = np.random.randint(0, 512);
#             y = np.random.randint(0, 384);
#             prev_angle = np.random.random() * np.pi * 2;
#         else:
#             angle = prev_angle + time_dist * momentum_predictions[i][1] * (-0.5 + np.random.random());
#             cnt = 0;
#             nx = x + np.cos(angle) * dist_len;
#             ny = y + np.sin(angle) * dist_len;
#             while cnt < 20 and (nx < 0 or nx > 512 or ny < 0 or ny > 384):
#                 cnt = cnt+1;
#                 angle = prev_angle + time_dist * momentum_predictions[i][1] * cnt * (-0.5 + np.random.random());
#                 nx = x + np.cos(angle) * dist_len;
#                 ny = y + np.sin(angle) * dist_len;
#             if nx < 0 or nx > 512 or ny < 0 or ny > 384:
#                 nx = np.random.randint(0, 512);
#                 ny = np.random.randint(0, 384);
#             x = nx;
#             y = ny;
#             prev_angle = angle;
#         print("{},{},{},1,0,0:0:0".format(int(x), int(y), int(timestamps[i])))
#         prev_timestamp = timestamps[i];

In [9]:
# Save the rhythm data!

np.savez_compressed("rhythm_data", objs = is_obj_pred[:, 0], predictions = another_pred_result, timestamps = timestamps, ticks = ticks, momenta = momentum_predictions, sv = (div_data[:,6] + 1) * 150);